In [30]:
import os
import sys
import numpy as np
import cv2
import lungs_finder as lf
from tqdm import tqdm

def scan(DATA_PATH, to_directory):
    i=0
    pole1=0
    pole2=0
    pole3=0
    for filename in tqdm(os.listdir(DATA_PATH)):
        path = os.path.join(DATA_PATH,filename)
        image = cv2.imread(path, 0)
        image = cv2.equalizeHist(image)
        image = cv2.medianBlur(image, 3)


        scaled_image = image# cv2.resize(image, (1024,1024))
        right_lung_hog_rectangle = lf.find_right_lung_hog(scaled_image)
        left_lung_hog_rectangle = lf.find_left_lung_hog(scaled_image)
        right_lung_lbp_rectangle = lf.find_right_lung_lbp(scaled_image)
        left_lung_lbp_rectangle = lf.find_left_lung_lbp(scaled_image)
        right_lung_haar_rectangle = lf.find_right_lung_haar(scaled_image)
        left_lung_haar_rectangle = lf.find_left_lung_haar(scaled_image)
        color_image = cv2.cvtColor(scaled_image, cv2.COLOR_GRAY2BGR)

        metoda = 0

        if (right_lung_hog_rectangle is not None) and (left_lung_hog_rectangle is not None) :
            x1, y1, width1, height1 = right_lung_hog_rectangle
            x2, y2, width2, height2 = left_lung_hog_rectangle
            cimage1 = scaled_image[np.minimum(y1, y2):np.maximum(y1+height1, y2+height2),x1:x2+width2] 
            pole1 = width1*height1+width2*height2
            metoda = 1 

        if (right_lung_lbp_rectangle is not None) and (left_lung_lbp_rectangle is not None):
            x1, y1, width1, height1 = right_lung_lbp_rectangle
            x2, y2, width2, height2 = left_lung_lbp_rectangle              
            cimage2 = scaled_image[np.minimum(y1, y2):np.maximum(y1+height1, y2+height2),x1:x2+width2] 
            pole2 = width1*height1+width2*height2
            metoda = 2

        if (right_lung_haar_rectangle is not None) and (left_lung_haar_rectangle is not None):
            x1, y1, width1, height1 = right_lung_haar_rectangle
            x2, y2, width2, height2 = left_lung_haar_rectangle
            cimage3 = scaled_image[np.minimum(y1, y2):np.maximum(y1+height1, y2+height2),x1:x2+width2] 
            pole3 = width1*height1+width2*height2
            metoda = 3

        #cv2.imshow("lungs-finder", cimage)
        if (pole1 >= pole2) and (pole1 >= pole3) and (pole1>0):
            cimage = cimage1
        if (pole2 > pole1) and (pole2 >= pole3):
            cimage = cimage2
        if (pole3 > pole1) and (pole3 > pole2):
            cimage = cimage3
        if (cimage.shape[0]>0) and (cimage.shape[1]>0):
            cropped = cv2.resize(cimage, (224,224))
            path_to_save = to_directory
            im_name=str(i)
            im_path = os.path.join(path_to_save, f'{im_name}.jpg') 
            cv2.imwrite(im_path, cropped) 
            i=i+1



In [36]:
from_directory = 'H:/Projekty/Dane/COVID-19 Radiography Database/pneumonia/not_cropped/class_c'
to_directory = 'H:/Projekty/Dane/COVID-19 Radiography Database/pneumonia/cropped/class_c'
scan(from_directory, to_directory)

100%|██████████████████████████████████████████████████████████████████████████████| 1345/1345 [06:24<00:00,  3.50it/s]


In [33]:
#!pip install lungs-finder

'H:\\Projekty\\Dane\\COVID-19 Radiography Database\\jedno\\1.jpg'